# Terremotos en Mexico

Proceso ETL:

+ Extraccion de datos desde el servicio sismologico nacional de la UNAM: http://www.ssn.unam.mx
+ Transformacion necesaria de los datos
+ Carga de datos en base de datos y mapa.

In [1]:
import warnings
warnings.simplefilter('ignore')

import requests as req
import xmltodict
import re

import pandas as pd

**Extraccion**

In [4]:
url='http://www.ssn.unam.mx/rss/ultimos-sismos.xml'

data=req.get(url).content

In [14]:
xmltodict.parse(data)['rss']['channel']['item'][0]

OrderedDict([('title', '4.3, 92 km al SUROESTE de  TONALA, CHIS'),
             ('description',
              '<p>Fecha:2020-12-14 11:23:30 (Hora de M&eacute;xico)<br/>Lat/Lon: 15.35/-94.15<br/>Profundidad: 37.0 km </p>'),
             ('link',
              'http://www2.ssn.unam.mx:8080/jsp/localizacion-de-sismo.jsp?latitud=15.35&longitud=-94.15&prf=37.0 km&ma=4.3&fecha=2020-12-14&hora=11:23:30&loc=92 km al SUROESTE de  TONALA, CHIS        &evento=1'),
             ('geo:lat', '15.35'),
             ('geo:long', '-94.15')])

In [17]:
len(xmltodict.parse(data)['rss']['channel']['item'])

15

In [15]:
xmltodict.parse(data)['rss']['channel']['item'][0]['title']

'4.3, 92 km al SUROESTE de  TONALA, CHIS'

In [16]:
def get_data():
    
    url='http://www.ssn.unam.mx/rss/ultimos-sismos.xml'

    data=req.get(url).content
    
    return xmltodict.parse(data)

**Transformacion**

In [55]:
def get_dictio():
    
    d=get_data()
    
    res=[]
    
    lst=d['rss']['channel']['item']
    
    for e in lst:
        
        link=e['link']
        
        fecha=re.findall('fecha=\d+-\d+-\d+', link)[0].split('=')[1]
        hora=re.findall('hora=\d+:\d+:\d+', link)[0].split('=')[1]
        lat=re.findall('latitud=\d+\.\d+', link)[0].split('=')[1]
        lng=re.findall('longitud=-\d+\.\d+', link)[0].split('=')[1]
        prof=re.findall('prf=\d+\.\d+', link)[0].split('=')[1]
        inten=float(re.findall('ma=\d+\.\d+', link)[0].split('=')[1])
        
        data={'Fecha': fecha, 'Hora': hora, 'Latitud':lat,
              'Longitud': lng, 'Profundidad':prof, 'Intensidad':inten}
        
        res.append(data)
        
    return res
        

In [56]:
get_dictio()[0]

{'Fecha': '2020-12-14',
 'Hora': '11:23:30',
 'Latitud': '15.35',
 'Longitud': '-94.15',
 'Profundidad': '37.0',
 'Intensidad': 4.3}

In [57]:
lst=xmltodict.parse(data)['rss']['channel']['item'][0]['link'].split('?')[1].split('&')

for e in lst:
    print(e.split('='))

['latitud', '15.35']
['longitud', '-94.15']
['prf', '37.0 km']
['ma', '4.3']
['fecha', '2020-12-14']
['hora', '11:23:30']
['loc', '92 km al SUROESTE de  TONALA, CHIS        ']
['evento', '1']


In [58]:
from urllib.parse import urlparse
u=urlparse(xmltodict.parse(data)['rss']['channel']['item'][0]['link'])
u.query

'latitud=15.35&longitud=-94.15&prf=37.0 km&ma=4.3&fecha=2020-12-14&hora=11:23:30&loc=92 km al SUROESTE de  TONALA, CHIS        &evento=1'

**Carga base datos de mongo**

**Mapa (heatmap)**

**Heatmap animado**